<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D0%BE%D0%B2_%D0%B8_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%BE%D0%B2_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D1%8B%D1%85_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9_%D0%B4%D0%BB%D1%8F_%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D0%B8_%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых библиотек
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets torch scikit-learn matplotlib seaborn

In [ ]:
# Импорт библиотек
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

# Загрузка датасета SQuAD
dataset = load_dataset("squad")

# Создание нового DatasetDict с первыми 100 записями
small_dataset = DatasetDict({
    'train': dataset['train'].select(range(50)),
    'validation': dataset['validation'].select(range(50))
})

# Выбор предобученной модели (например, BERT)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Функция для токенизации данных
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Нахождение токенов, соответствующих началу и концу ответа
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx

        if start_char == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            token_start_index = context_start
            token_end_index = len(inputs.input_ids[i]) - 1

            while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while token_end_index >= 0 and offset[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Применение функции предобработки к уменьшенному датасету
tokenized_datasets = small_dataset.map(preprocess_function, batched=True, remove_columns=small_dataset["train"].column_names)

# Загрузка предобученной модели для QA
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Определение функции вычисления метрик
def compute_metrics(p):
    start_logits, end_logits = p.predictions
    start_pred = np.argmax(start_logits, axis=1)
    end_pred = np.argmax(end_logits, axis=1)
    start_true = p.label_ids[:, 0]
    end_true = p.label_ids[:, 1]

    exact_match = np.mean((start_true == start_pred) & (end_true == end_pred))
    f1 = (f1_score(start_true, start_pred, average="macro") + f1_score(end_true, end_pred, average="macro")) / 2
    return {"exact_match": exact_match, "f1": f1}

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Отключение W&B
)

# Создание объекта Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

# Визуализация результатов
metrics = trainer.state.log_history

# График потерь
train_loss = [m["loss"] for m in metrics if "loss" in m]
eval_loss = [m["eval_loss"] for m in metrics if "eval_loss" in m]

plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Train Loss")
plt.plot(eval_loss, label="Validation Loss")
plt.title("Loss during training")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.show()

# График метрик
exact_match = [m["eval_exact_match"] for m in metrics if "eval_exact_match" in m]
f1_scores = [m["eval_f1"] for m in metrics if "eval_f1" in m]

plt.figure(figsize=(10, 5))
plt.plot(exact_match, label="Exact Match")
plt.plot(f1_scores, label="F1 Score")
plt.title("Metrics during training")
plt.xlabel("Epochs")
plt.ylabel("Score")
plt.legend()
plt.show()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-27-67b69a33a905>:109: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
# Функция для предсказания ответа на новый вопрос
def predict_answer(question, context):
    # Токенизация вопроса и контекста
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",  # Возвращаем PyTorch тензоры
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True  # Возвращаем отображение токенов в исходный текст
    )

    # Получение выходных данных модели
    with torch.no_grad():  # Отключение вычисления градиентов (так как это инференс)
        outputs = model(**inputs)

    # Определение позиций начала и конца ответа
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_index = torch.argmax(start_logits, dim=1).item()  # Индекс начала ответа
    end_index = torch.argmax(end_logits, dim=1).item()      # Индекс конца ответа

    # Преобразование индексов токенов в оригинальный текст
    offsets = inputs["offset_mapping"][0].numpy()
    start_char = offsets[start_index][0]
    end_char = offsets[end_index][1]

    # Извлечение ответа из контекста
    answer = context[start_char:end_char]

    return answer

# Пример тестирования модели на новых данных
question = "What is the capital of France?"
context = "France, officially the French Republic, is a country primarily located in Western Europe. Its capital is Paris."

# Предсказание ответа
answer = predict_answer(question, context)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

# Список тестовых примеров
test_cases = [
    {
        "question": "What is the largest planet in our solar system?",
        "context": "Jupiter is the largest planet in our solar system. It has a mass of about 1/1000 that of the Sun.",
        "expected_answer": "Jupiter"
    },
    {
        "question": "Who wrote the play 'Hamlet'?",
        "context": "William Shakespeare was an English playwright and poet. He wrote many famous plays, including 'Hamlet'.",
        "expected_answer": "William Shakespeare"
    }
]

# Проверка модели на тестовых примерах
for i, test_case in enumerate(test_cases):
    question = test_case["question"]
    context = test_case["context"]
    expected_answer = test_case["expected_answer"]

    # Предсказание ответа
    answer = predict_answer(question, context)

    # Вывод результатов
    print(f"\nTest Case {i + 1}:")
    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Predicted Answer: {answer}")

    # Проверка корректности ответа
    if expected_answer.lower() in answer.lower():
        print("Result: Correct")
    else:
        print("Result: Incorrect")

TypeError: BertForQuestionAnswering.forward() got an unexpected keyword argument 'offset_mapping'